<a href="https://colab.research.google.com/github/GuilhermeDeAssis123/DataScience/blob/master/Previs%C3%A3o_de_Altas_e_Obitos_Relacionadas_ao_COVID_19_com_Dados_de_Espectrometria.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>




# <font color = 'red'> Previsão de Altas e Obitos Relacionadas ao COVID_19 com Dados de Espectrometria. </font> 

> Já faz mais de um ano que o Brasil se encontra em estado pandemico, trazendo inumeros prejuizos economicos a nosso pais, sem falar nas incontaveis mortes e individuos infectados pelo novo corona virus e suas variantes.


<center>

  <img src = 'https://www2.ufjf.br/progepe/wp-content/uploads/sites/28/2020/03/Covid-19-1024x576.jpg'/>

</center>

> Aqui no Brasil ja alcançamos uma marca superior a 11 milhões de pessoas infectadas e mais de 290 mil vitimas pelo novo corona virus, como podemos ver, esses são numeros de um crescimento desemfreado em relação ao avanço da doença.

> Por isso neste notebook irei realizar a criação de um simples modelo que preveja uma situação de alta ou obtio por parte dos infectados de acordo com as moleculas que estão presentes no corpo do paciente e levando em consideração também a intensidade dessa presença (quantidade presente).


## <font color = 'red'> 1. Obtenção dos Dados: </font>

> Os dados que serão utilizados para a realização desse notebook foram retirados do seguinte [link](https://zenodo.org/record/4329382#.YFYqcq9KhPZ). Esse dados foram disponibilizados por um projeto da UNICAMP em função de dispor os resultados de um pesquisa também relacionada ao coronavirus. Por aqui vamos trabalhar com o arquivo que se encontra na pasta `Covid_Control_X_LowRisk`, o arquivo que foi setado como o arquivo de `Fitting` para o modelo.

## <font color = 'red'> 2. Entendendo os Dados: <font>

> Nesta seção vamos explicar um pouco sobre os dados que estamos lidando e o que de fato significam as *features* desse *dataset*. Como citado no titulo deste notebook, aqui vamos trabalhar com dados de espectrometria de massa. Caso queira entender mais a fundo esse tipo de dado, vou disponibilizar um [link](https://www.news-medical.net/life-sciences/What-is-Mass-Spectrometry-(Portuguese).aspx) para que o leitor possa se aprofundar um pouco mais. Porém de uma forma resumida, a espctrometria de massa nada mais é do que realizar um *scanner* de um substancia atravéz de um espectrometro, isso por sua vez irá retornar as moleculas presentes naquela substancia e a quantidade de cada molecula, assim forncendo as informações do que contém em seu sangue por exemplo.

 ### <font color = 'red'> 2.1 Dicionario de Dados </font>
> Nesta seção vou descrever de forma breve as colunas presentes no nosso *dataset*:

* `Colunas`: As colunas, que são do tipo `Covid19_XXXXX_XX` diz respeito a um espectro especifico de um paciente, logo, cada coluna representa um espectro de um paciente. No qual o primeiro conjunto de número corresponde ao numero do paciente, e o segundo conjunto à identificação do espectro recolhido do paciente.

* `Index_0(# individual)`: Essa linha corresponde a indetificação do paciente, como se fosse um nome

* `Index_1(# indiv number)`: Essa linha corresponde a um número de identificação do paciente.

* `Index_2(# obsv numberl)`: Diz respeito ao número de identificação do espectro recolhido, logo o número 3, diz que essa foi a terceira espectrometria em cima daquele paciente.

* `Index_3(# categ number)`: Aqui o dado informa a qual grupo aquele paciente pertence, de modo que os grupos são:
  * Grupos 1, 2, 3, 4, 5, 6 e 7 --> são grupos de controle.
  * Grupos 8 e 12 --> são grupos que possuem a doença de forma leve
  * Grupos 9, 10, 11, 13 e 14 --> são grupos que possuem a doença de forma grave.

* `Index_4(# class)`: Aqui é o nosso alvo, essa informação indica se ouve obito ou alta por parte do paciente, se for 0, teve alta. Se for 1, o paciente foi a obito.

* `Index_5 em diante`: Aqui os indices passam a ser do tipo ``XXX_XXX`, isso indica uma molecula, de modo que cada numero indica uma molecula diferente.

## <font color = 'red'> 3. Importação dos Dados: <font>

> Nesta estapa vamos verificar qual a cara do nosso conjunto de dados, formato, composição.

> Minha hipótese inicial é de que seja um conjunto de dados bem esparso uma vez que dificilmente os paciente tenham as mesmas moleculas ou as mesmas quantidades. Como visto no dicionarios de dados algumas colunas são do tipo descritvas e esse tipo de dado não passa pelo modelo e posteriormente terão de ser retiradas.

> Para começar o primeiro passo é importar as bibliotecas que serão utilizadas no projeto.

In [ ]:
# biblioteca de tratamento dos dados
import pandas as pd
import numpy as np

#bibliotecas de importação do modelo
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression

#metricas de avaliação do modelo
from sklearn.metrics import roc_auc_score, roc_curve, accuracy_score
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix

#metricas de treinamento do modelo
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedKFold

#bibliotecas de visualização de dados
import seaborn as sns
import matplotlib.pyplot as plt

#filtrar avisos
import warnings
warnings.filterwarnings('ignore')

In [ ]:
#importação do arquivo de manipulação
df = pd.read_csv('/content/FeatureVectors_Cov19_Control_x_LowRisk_Fitting.csv')

In [ ]:
# Dimensões do dataframe
df.shape

(3962, 3798)

> Como podemos observar estamos lidando com um *dataset* que possui 3962 linhas por 3798 colunas.

In [ ]:
df.head(10)

,# vector name,Covid19_00715_01,Covid19_00715_02,Covid19_00715_03,Covid19_00715_04,Covid19_00715_05,Covid19_00715_06,Covid19_00715_07,Covid19_00715_08,Covid19_00715_09,Covid19_00715_10,Covid19_00716_01,Covid19_00716_02,Covid19_00716_03,Covid19_00716_04,Covid19_00716_05,Covid19_00716_06,Covid19_00716_07,Covid19_00716_08,Covid19_00716_09,Covid19_00716_10,Covid19_00717_01,Covid19_00717_02,Covid19_00717_03,Covid19_00717_04,Covid19_00717_05,Covid19_00717_06,Covid19_00717_07,Covid19_00717_08,Covid19_00717_09,Covid19_00717_10,Covid19_00719_01,Covid19_00719_02,Covid19_00719_03,Covid19_00719_04,Covid19_00719_05,Covid19_00719_06,Covid19_00719_07,Covid19_00719_08,Covid19_00719_09,...,Covid19_01061_01,Covid19_01061_02,Covid19_01061_03,Covid19_01061_04,Covid19_01061_05,Covid19_01061_06,Covid19_01061_07,Covid19_01061_08,Covid19_01061_09,Covid19_01061_10,Covid19_01062_01,Covid19_01062_02,Covid19_01062_03,Covid19_01062_04,Covid19_01062_05,Covid19_01062_06,Covid19_01062_07,Covid19_01062_08,Covid19_01062_09,Covid19_01062_10,Covid19_01065_01,Covid19_01065_02,Covid19_01065_03,Covid19_01065_04,Covid19_01065_05,Covid19_01065_06,Covid19_01065_07,Covid19_01065_08,Covid19_01065_09,Covid19_01065_10,Covid19_01090_01,Covid19_01090_02,Covid19_01090_03,Covid19_01090_04,Covid19_01090_05,Covid19_01090_06,Covid19_01090_07,Covid19_01090_08,Covid19_01090_09,Covid19_01090_10
0,# individual,Covid19_00715,Covid19_00715,Covid19_00715,Covid19_00715,Covid19_00715,Covid19_00715,Covid19_00715,Covid19_00715,Covid19_00715,Covid19_00715,Covid19_00716,Covid19_00716,Covid19_00716,Covid19_00716,Covid19_00716,Covid19_00716,Covid19_00716,Covid19_00716,Covid19_00716,Covid19_00716,Covid19_00717,Covid19_00717,Covid19_00717,Covid19_00717,Covid19_00717,Covid19_00717,Covid19_00717,Covid19_00717,Covid19_00717,Covid19_00717,Covid19_00719,Covid19_00719,Covid19_00719,Covid19_00719,Covid19_00719,Covid19_00719,Covid19_00719,Covid19_00719,Covid19_00719,...,Covid19_01061,Covid19_01061,Covid19_01061,Covid19_01061,Covid19_01061,Covid19_01061,Covid19_01061,Covid19_01061,Covid19_01061,Covid19_01061,Covid19_01062,Covid19_01062,Covid19_01062,Covid19_01062,Covid19_01062,Covid19_01062,Covid19_01062,Covid19_01062,Covid19_01062,Covid19_01062,Covid19_01065,Covid19_01065,Covid19_01065,Covid19_01065,Covid19_01065,Covid19_01065,Covid19_01065,Covid19_01065,Covid19_01065,Covid19_01065,Covid19_01090,Covid19_01090,Covid19_01090,Covid19_01090,Covid19_01090,Covid19_01090,Covid19_01090,Covid19_01090,Covid19_01090,Covid19_01090
1,# indiv number,1,1,1,1,1,1,1,1,1,1,2,2,2,2,2,2,2,2,2,2,3,3,3,3,3,3,3,3,3,3,4,4,4,4,4,4,4,4,4,...,377,377,377,377,377,377,377,377,377,377,378,378,378,378,378,378,378,378,378,378,379,379,379,379,379,379,379,379,379,379,380,380,380,380,380,380,380,380,380,380
2,# obsv number,1,2,3,4,5,6,7,8,9,10,1,2,3,4,5,6,7,8,9,10,1,2,3,4,5,6,7,8,9,10,1,2,3,4,5,6,7,8,9,...,1,2,3,4,5,6,7,8,9,10,1,2,3,4,5,6,7,8,9,10,1,2,3,4,5,6,7,8,9,10,1,2,3,4,5,6,7,8,9,10
3,# categ number,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,...,12,12,12,12,12,12,12,12,12,12,12,12,12,12,12,12,12,12,12,12,12,12,12,12,12,12,12,12,12,12,12,12,12,12,12,12,12,12,12,12
4,# class,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1
5,200.014,351466,342524,291625,319733,328780,286064,318682,363598,374650,351727,99172,106255,139807,162968,151349,154162,185156,176275,190065,175690,111510,118743,120583,145137,153634,141446,174804,172191,181676,181102,114579,119242,149046,157075,186594,156193,155802,186950,174104,...,86208,141545,153260,171418,195197,175391,195427,163256,180677,201931,103602,118295,138011,136765,148426,136539,135258,136557,155038,153466,0,100225,139982,135005,178649,163083,182002,209292,205180,203998,65085,92415,114840,121065,113422,136886,149557,169740,171250,167563
6,200.164,166428,0,72728,65305,65659,0,65943,75328,0,75061,109557,107664,132533,186609,99503,155498,136906,142950,115774

## <font color = 'red'> 4. Breve Exploração dos Dados: <font>

> Nesta seção vamos verificar se são necessárias algumas mudança e tratamentos nos dados. Como pudemos averiguar na célula de código anterior, o conjunto de dados é realmente muito esparso, talvez isso interfira no desempenho do modelo posteriormente.

> Vamos agora verificar se existem dados nulos:

In [ ]:
#Porcentagem de valores nulos
(df.isnull().sum() / df.shape[0]).sort_values(ascending = False)

Covid19_01090_10    0.003029
Covid19_00650_01    0.003029
Covid19_00652_03    0.003029
Covid19_00652_02    0.003029
Covid19_00652_01    0.003029
                      ...   
Covid19_00820_10    0.003029
Covid19_00820_09    0.003029
Covid19_00820_08    0.003029
Covid19_00820_07    0.003029
# vector name       0.003029
Length: 3798, dtype: float64

> De alguma maneira estranha todas as colunas possuem a mesma quantidade de dados nulos, isso talvez possa ser explicado por uma falha do espectrometro no momento de recolhimento dos dados.

> Vamos olhar mais de perto quais são essas linhas.

In [ ]:
#quais linhas são nulas?
df[df['Covid19_01090_10'].isnull()]

,# vector name,Covid19_00715_01,Covid19_00715_02,Covid19_00715_03,Covid19_00715_04,Covid19_00715_05,Covid19_00715_06,Covid19_00715_07,Covid19_00715_08,Covid19_00715_09,Covid19_00715_10,Covid19_00716_01,Covid19_00716_02,Covid19_00716_03,Covid19_00716_04,Covid19_00716_05,Covid19_00716_06,Covid19_00716_07,Covid19_00716_08,Covid19_00716_09,Covid19_00716_10,Covid19_00717_01,Covid19_00717_02,Covid19_00717_03,Covid19_00717_04,Covid19_00717_05,Covid19_00717_06,Covid19_00717_07,Covid19_00717_08,Covid19_00717_09,Covid19_00717_10,Covid19_00719_01,Covid19_00719_02,Covid19_00719_03,Covid19_00719_04,Covid19_00719_05,Covid19_00719_06,Covid19_00719_07,Covid19_00719_08,Covid19_00719_09,...,Covid19_01061_01,Covid19_01061_02,Covid19_01061_03,Covid19_01061_04,Covid19_01061_05,Covid19_01061_06,Covid19_01061_07,Covid19_01061_08,Covid19_01061_09,Covid19_01061_10,Covid19_01062_01,Covid19_01062_02,Covid19_01062_03,Covid19_01062_04,Covid19_01062_05,Covid19_01062_06,Covid19_01062_07,Covid19_01062_08,Covid19_01062_09,Covid19_01062_10,Covid19_01065_01,Covid19_01065_02,Covid19_01065_03,Covid19_01065_04,Covid19_01065_05,Covid19_01065_06,Covid19_01065_07,Covid19_01065_08,Covid19_01065_09,Covid19_01065_10,Covid19_01090_01,Covid19_01090_02,Covid19_01090_03,Covid19_01090_04,Covid19_01090_05,Covid19_01090_06,Covid19_01090_07,Covid19_01090_08,Covid19_01090_09,Covid19_01090_10
3950,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3951,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3952,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3953,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3954,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3955,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3956,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3957,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3958,N

> Observa-se que das linhas 3950 a 3961 todas as colunas possuem valores nulos. Como os valores de espectrometria são um pouco dificeis de serem imputados devido a especificidade dado, vamos optar por remover essas linhas.

In [ ]:
# Removendo linhas com os valores NaN
df.dropna(inplace=True)

In [ ]:
# Mostrando dados ausentes
(df.isnull().sum() / df.shape[0]).sort_values(ascending = False)

Covid19_01090_10    0.0
Covid19_00650_01    0.0
Covid19_00652_03    0.0
Covid19_00652_02    0.0
Covid19_00652_01    0.0
                   ... 
Covid19_00820_10    0.0
Covid19_00820_09    0.0
Covid19_00820_08    0.0
Covid19_00820_07    0.0
# vector name       0.0
Length: 3798, dtype: float64

> Como podemos ver agora na célula anterior não existem mais dados nulos.

In [ ]:
df.head()

,# vector name,Covid19_00715_01,Covid19_00715_02,Covid19_00715_03,Covid19_00715_04,Covid19_00715_05,Covid19_00715_06,Covid19_00715_07,Covid19_00715_08,Covid19_00715_09,Covid19_00715_10,Covid19_00716_01,Covid19_00716_02,Covid19_00716_03,Covid19_00716_04,Covid19_00716_05,Covid19_00716_06,Covid19_00716_07,Covid19_00716_08,Covid19_00716_09,Covid19_00716_10,Covid19_00717_01,Covid19_00717_02,Covid19_00717_03,Covid19_00717_04,Covid19_00717_05,Covid19_00717_06,Covid19_00717_07,Covid19_00717_08,Covid19_00717_09,Covid19_00717_10,Covid19_00719_01,Covid19_00719_02,Covid19_00719_03,Covid19_00719_04,Covid19_00719_05,Covid19_00719_06,Covid19_00719_07,Covid19_00719_08,Covid19_00719_09,...,Covid19_01061_01,Covid19_01061_02,Covid19_01061_03,Covid19_01061_04,Covid19_01061_05,Covid19_01061_06,Covid19_01061_07,Covid19_01061_08,Covid19_01061_09,Covid19_01061_10,Covid19_01062_01,Covid19_01062_02,Covid19_01062_03,Covid19_01062_04,Covid19_01062_05,Covid19_01062_06,Covid19_01062_07,Covid19_01062_08,Covid19_01062_09,Covid19_01062_10,Covid19_01065_01,Covid19_01065_02,Covid19_01065_03,Covid19_01065_04,Covid19_01065_05,Covid19_01065_06,Covid19_01065_07,Covid19_01065_08,Covid19_01065_09,Covid19_01065_10,Covid19_01090_01,Covid19_01090_02,Covid19_01090_03,Covid19_01090_04,Covid19_01090_05,Covid19_01090_06,Covid19_01090_07,Covid19_01090_08,Covid19_01090_09,Covid19_01090_10
0,# individual,Covid19_00715,Covid19_00715,Covid19_00715,Covid19_00715,Covid19_00715,Covid19_00715,Covid19_00715,Covid19_00715,Covid19_00715,Covid19_00715,Covid19_00716,Covid19_00716,Covid19_00716,Covid19_00716,Covid19_00716,Covid19_00716,Covid19_00716,Covid19_00716,Covid19_00716,Covid19_00716,Covid19_00717,Covid19_00717,Covid19_00717,Covid19_00717,Covid19_00717,Covid19_00717,Covid19_00717,Covid19_00717,Covid19_00717,Covid19_00717,Covid19_00719,Covid19_00719,Covid19_00719,Covid19_00719,Covid19_00719,Covid19_00719,Covid19_00719,Covid19_00719,Covid19_00719,...,Covid19_01061,Covid19_01061,Covid19_01061,Covid19_01061,Covid19_01061,Covid19_01061,Covid19_01061,Covid19_01061,Covid19_01061,Covid19_01061,Covid19_01062,Covid19_01062,Covid19_01062,Covid19_01062,Covid19_01062,Covid19_01062,Covid19_01062,Covid19_01062,Covid19_01062,Covid19_01062,Covid19_01065,Covid19_01065,Covid19_01065,Covid19_01065,Covid19_01065,Covid19_01065,Covid19_01065,Covid19_01065,Covid19_01065,Covid19_01065,Covid19_01090,Covid19_01090,Covid19_01090,Covid19_01090,Covid19_01090,Covid19_01090,Covid19_01090,Covid19_01090,Covid19_01090,Covid19_01090
1,# indiv number,1,1,1,1,1,1,1,1,1,1,2,2,2,2,2,2,2,2,2,2,3,3,3,3,3,3,3,3,3,3,4,4,4,4,4,4,4,4,4,...,377,377,377,377,377,377,377,377,377,377,378,378,378,378,378,378,378,378,378,378,379,379,379,379,379,379,379,379,379,379,380,380,380,380,380,380,380,380,380,380
2,# obsv number,1,2,3,4,5,6,7,8,9,10,1,2,3,4,5,6,7,8,9,10,1,2,3,4,5,6,7,8,9,10,1,2,3,4,5,6,7,8,9,...,1,2,3,4,5,6,7,8,9,10,1,2,3,4,5,6,7,8,9,10,1,2,3,4,5,6,7,8,9,10,1,2,3,4,5,6,7,8,9,10
3,# categ number,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,...,12,12,12,12,12,12,12,12,12,12,12,12,12,12,12,12,12,12,12,12,12,12,12,12,12,12,12,12,12,12,12,12,12,12,12,12,12,12,12,12
4,# class,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1


> Agora vamos fazer algumas manobras com os dados para que fique mais fácil de visualizar a informação útil para o modelo. Vamos fazer removendo algumas colunas e reposicionando os dados trocando o índice e utilizando a técnica de transposição.

In [ ]:
# Resetando o index para a coluna vector name 
df.index = df['# vector name']

# Dropando a duplicata 
df.drop(columns=['# vector name'], axis=1)

,Covid19_00715_01,Covid19_00715_02,Covid19_00715_03,Covid19_00715_04,Covid19_00715_05,Covid19_00715_06,Covid19_00715_07,Covid19_00715_08,Covid19_00715_09,Covid19_00715_10,Covid19_00716_01,Covid19_00716_02,Covid19_00716_03,Covid19_00716_04,Covid19_00716_05,Covid19_00716_06,Covid19_00716_07,Covid19_00716_08,Covid19_00716_09,Covid19_00716_10,Covid19_00717_01,Covid19_00717_02,Covid19_00717_03,Covid19_00717_04,Covid19_00717_05,Covid19_00717_06,Covid19_00717_07,Covid19_00717_08,Covid19_00717_09,Covid19_00717_10,Covid19_00719_01,Covid19_00719_02,Covid19_00719_03,Covid19_00719_04,Covid19_00719_05,Covid19_00719_06,Covid19_00719_07,Covid19_00719_08,Covid19_00719_09,Covid19_00719_10,...,Covid19_01061_01,Covid19_01061_02,Covid19_01061_03,Covid19_01061_04,Covid19_01061_05,Covid19_01061_06,Covid19_01061_07,Covid19_01061_08,Covid19_01061_09,Covid19_01061_10,Covid19_01062_01,Covid19_01062_02,Covid19_01062_03,Covid19_01062_04,Covid19_01062_05,Covid19_01062_06,Covid19_01062_07,Covid19_01062_08,Covid19_01062_09,Covid19_01062_10,Covid19_01065_01,Covid19_01065_02,Covid19_01065_03,Covid19_01065_04,Covid19_01065_05,Covid19_01065_06,Covid19_01065_07,Covid19_01065_08,Covid19_01065_09,Covid19_01065_10,Covid19_01090_01,Covid19_01090_02,Covid19_01090_03,Covid19_01090_04,Covid19_01090_05,Covid19_01090_06,Covid19_01090_07,Covid19_01090_08,Covid19_01090_09,Covid19_01090_10
# vector name,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
# individual,Covid19_00715,Covid19_00715,Covid19_00715,Covid19_00715,Covid19_00715,Covid19_00715,Covid19_00715,Covid19_00715,Covid19_00715,Covid19_00715,Covid19_00716,Covid19_00716,Covid19_00716,Covid19_00716,Covid19_00716,Covid19_00716,Covid19_00716,Covid19_00716,Covid19_00716,Covid19_00716,Covid19_00717,Covid19_00717,Covid19_00717,Covid19_00717,Covid19_00717,Covid19_00717,Covid19_00717,Covid19_00717,Covid19_00717,Covid19_00717,Covid19_00719,Covid19_00719,Covid19_00719,Covid19_00719,Covid19_00719,Covid19_00719,Covid19_00719,Covid19_00719,Covid19_00719,Covid19_00719,...,Covid19_01061,Covid19_01061,Covid19_01061,Covid19_01061,Covid19_01061,Covid19_01061,Covid19_01061,Covid19_01061,Covid19_01061,Covid19_01061,Covid19_01062,Covid19_01062,Covid19_01062,Covid19_01062,Covid19_01062,Covid19_01062,Covid19_01062,Covid19_01062,Covid19_01062,Covid19_01062,Covid19_01065,Covid19_01065,Covid19_01065,Covid19_01065,Covid19_01065,Covid19_01065,Covid19_01065,Covid19_01065,Covid19_01065,Covid19_01065,Covid19_01090,Covid19_01090,Covid19_01090,Covid19_01090,Covid19_01090,Covid19_01090,Covid19_01090,Covid19_01090,Covid19_01090,Covid19_01090
# indiv number,1,1,1,1,1,1,1,1,1,1,2,2,2,2,2,2,2,2,2,2,3,3,3,3,3,3,3,3,3,3,4,4,4,4,4,4,4,4,4,4,...,377,377,377,377,377,377,377,377,377,377,378,378,378,378,378,378,378,378,378,378,379,379,379,379,379,379,379,379,379,379,380,380,380,380,380,380,380,380,380,380
# obsv number,1,2,3,4,5,6,7,8,9,10,1,2,3,4,5,6,7,8,9,10,1,2,3,4,5,6,7,8,9,10,1,2,3,4,5,6,7,8,9,10,...,1,2,3,4,5,6,7,8,9,10,1,2,3,4,5,6,7,8,9,10,1,2,3,4,5,6,7,8,9,10,1,2,3,4,5,6,7,8,9,10
# categ number,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,...,12,12,12,12,12,12,12,12,12,12,12,12,12,12,12,12,12,12,12,12,12,12,12,12,12,12,12,12,12,12,12,12,12,12,12,12,12,12,12,12
# class,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1594.17,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,159616,126356,135186,0
1595.18,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,

In [ ]:
# Realizando a matriz transposta dos dados
df = df.T

> Acima nos ralizamos a matriz transposta dos dados, utilizando essa técnica fazemos com que linhas virem as colunas e as colunas virem as linhas. Isso nos ajuda uma vez que, agora todas as linhas(entradas) representam um espectro de um paciente, e as colunas represetam os atributos(moleculas) daquele paciente.

In [ ]:
# Dropando colunas que não seram usadas pelo modelo.
df.drop(columns = ['# individual', '# indiv number', '# obsv number', '# categ number'], axis=1, inplace=True)

> Colunas retiradas, estão são as presentes na celula de código acima.

In [ ]:
#resetando o index para um valorer numerico
df.reset_index(drop= True,inplace=True)

In [ ]:
#Dropando a duplicata
df.drop(df.index[0], axis=0, inplace=True)

In [ ]:
df.head()

# vector name,# class,200.014,200.164,200.165,200.172,200.173,200.2005,200.201,200.974,200.9843,200.996,201.007,201.008,201.009,201.021,201.043,201.073,201.0735,201.112,201.148,201.149,201.184,201.185,201.9874,202.012,202.016,202.029,202.0329,202.0332,202.4096,202.4101,202.572,202.573,202.574,202.8737,202.8744,203,203.0002,203.0204,203.022,...,1295.96,1296.59,1296.96,1305.93,1388.84,1389.85,1394.87,1395.87,1396.88,1418.86,1516.13,1517.13,1518.14,1519.15,1520.15,1540.13,1541.13,1542.14,1543.15,1544.16,1545.16,1546.17,1547.18,1564.13,1565.13,1566.14,1567.15,1568.16,1569.16,1570.17,1571.18,1572.19,1573.19,1592.16,1593.16,1594.17,1595.18,1596.19,1610.94,1611.94
1,0,351466,166428,0,138399,0,113675,0,0,94225,0,831531,0,0,162277,103249,92731,0,117530,101465,0,0,0,0,325214,0,363577,358532,0,0,0,0,0,0,0,0,214327,0,960430,658917,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,0,342524,0,0,142524,0,158046,0,0,94445,0,730175,0,0,170644,83406,79405,0,117096,96035,0,60024,0,0,317292,0,453057,362187,0,0,0,0,0,0,0,0,213902,0,1096103,618202,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,0,291625,72728,0,185878,0,132406,0,0,70461,0,626350,0,0,149242,76336,85179,0,127489,103466,0,0,0,0,243723,0,316609,335303,0,0,0,0,0,0,0,0,224713,0,943616,493762,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,0,319733,65305,0,175918,0,114670,0,0,65818,0,588527,0,0,118403,83078,67896,0,105728,112666,0,67991,0,0,244880,0,254684,354174,0,0,0,0,0,0,0,0,239695,0,916437,511540,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
5,0,328780,65659,0,151264,0,131899,0,0,76439,0,573797,0,0,132134,74743,68168,0,117243,116233,0,60443,0,0,208479,0,235842,373589,0,0,0,0,0,0,0,0,271820,0,878546,487899,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


> Como pode-se observar, agora temos somente as colunas relevantes para o modelo. Vemos tambem que fica mais fácil de relacionar os dados visualmente na cabeça do leitor e entender a correlação entre os dados.

> Vamos agora verificar os tipos das colunas:

In [ ]:
df.dtypes.unique()

array([dtype('O')], dtype=object)

> Vemos que o único tipo presente é o tipo `object` ou `string`. Esses dados não passariam pelo modelo estando classificados dessa forma. Vamos então converter todas as colunas para `int` ou `float`

In [ ]:
# Convertendo colunas
for coluna in df:
  df[coluna] = pd.to_numeric(df[coluna])

In [ ]:
df.dtypes.unique()

array([dtype('int64'), dtype('float64')], dtype=object)

> Vemos agora que os unicos tipo presentes no *dataset* são `int` e `float`.

## <font color = 'red'> 5. Preparação e criação do modelo SVM: <font>

> Nesta etapa vamos primeiramente dividir nosso conjuntos dados em *dataset* de treino e teste. O *dataset* de treino sera utilizado durante todo o tempo para o ajuste de paramentro e melhora de desempenho do modelo. Enquanto que o *dataset* de teste será utilizado somente ao final para verificar o desempenho do modelo treinado.

In [ ]:
#Separação da coluna alvo do resto dos dados.
X = df.drop(['# class'],axis=1)
y = df['# class']

In [ ]:
#Separando os dados em treino e teste.
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3)

> Aqui vamos aprimorar os hiperparametros do nosso modelo. Porém isso pode ser uma tarefa muito demorada, de forma que vamos realizar ela em partes para agilizar o processo de treinamento. Para tal, vamos treinar os hiperparamentros de forma separada, um por um.

> Para isso, vamos utilizar o modelo de aprendizagem de máquina SVM(Sub Vetorial Machine) por opção de escolha minha, pois domindo mais esse modelo do que outros.

> Vamos setar os paramentros na seguinte ordem:

*  `Kernel` : Tipo de função de aproximação
*  `C` : Passo de aproximação

> A metrica que vamos buscar pelo melhor desempenho é o `f1-score`.

In [ ]:

svc = SVC(C=1)
param_grid = {
 'kernel':['rbf','poly','sigmoid','linear']
}

# identificar melhor parâmetro
kfold = StratifiedKFold(n_splits=5, shuffle=True)
grid_search = GridSearchCV(svc, param_grid, scoring='f1', n_jobs=-1, cv=kfold)
grid_result = grid_search.fit(X_train, y_train)

# ver resultados
print("Melhor: {} para {}".format(grid_result.best_score_, grid_result.best_params_))

Melhor: 0.9957026087356956 para {'kernel': 'linear'}


>Para setar a melhor função usamos arbitrariamente o passo de aproximação como 1. Assim descobrimos que a melhor função de aproximação, é uma **função linear** com um desempenho no treino de **99,77%**.

> Agora que ja sabemos a melhor função, vamos procurar pela melhor aproximação:

In [ ]:
svc = SVC(kernel='linear')

param_grid = {
 'C':[0.00001, 0.0001, 0.001, 0.01, 0.1, 1, 10, 100, 1000]
}

# identificar melhor parâmetro
kfold = StratifiedKFold(n_splits=5, shuffle=True)
grid_search = GridSearchCV(svc, param_grid, scoring='f1', n_jobs=-1, cv=kfold)
grid_result = grid_search.fit(X_train, y_train)

# ver resultados
print("Melhor: {} para {}".format(grid_result.best_score_, grid_result.best_params_))

Melhor: 0.9978531187505665 para {'C': 1e-05}


> Setando a função de aproximação para uma função linear e testando alguns passos de aproximação, vemos que o melhor foi com o valore de `C = 0.00001`, aumentando o desempenho do modelo para **99,79%**.

> Com os dois paramentros setados vamos agora treinar o modelo com os valores encontrados e ver seu desempenho:


In [ ]:
svc = SVC(C = 0.00001, kernel='linear')
svc.fit(X_train,y_train)

y_pred = svc.predict(X_test)

# Classification Report
print(classification_report(y_test, y_pred))

# imprimir a área sob a curva
print("AUC: {:.4f}\n".format(roc_auc_score(y_test, y_pred)))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00       607
           1       1.00      1.00      1.00       533

    accuracy                           1.00      1140
   macro avg       1.00      1.00      1.00      1140
weighted avg       1.00      1.00      1.00      1140

AUC: 0.9982



> Devido ao alto desempenho do modelo, vamos realizar a construção e treinamento de outros dois modelos para verificar se o desempenho adiquierido pelo modelo de SVM é plausível. Os modelos que iremos testar são:

* Regressão Logistica.

* Arvore de decisão.

Então vamos começar pelo modelo de Arvore de Decisão.

## <font color = 'red'> 5.1 Modelo de Arvore de Decisão : <font>

> Vamos setar alguns parametros afim de maximizar o desempenho de modelo, esse parametro são:

*  `max_depth`

* `criterion`

* `splitter`

> Aqui a hipótese incial é que o desempenho desse modelo não seja tão bom quanto o modelo de SVM, pois naturalmete o SVM desempenha melhores resultados que o modelo de arvore de decisão, mas aqui é esperado uma taxa de acerto por volta de 92%.

In [ ]:
dt = DecisionTreeClassifier()

param_grid = {
 'max_depth':[4,5,6,7,8]
}

# identificar melhor parâmetro
kfold = StratifiedKFold(n_splits=5, shuffle=True)
grid_search = GridSearchCV(dt, param_grid, scoring='f1', n_jobs=-1, cv=kfold)
grid_result = grid_search.fit(X_train, y_train)

# ver resultados
print("Melhor: {} para {}".format(grid_result.best_score_, grid_result.best_params_))



Melhor: 0.9524516670251758 para {'max_depth': 8}


In [ ]:
dt = DecisionTreeClassifier(max_depth=8)

param_grid = {
 'criterion':['gini','entropy']
}

# identificar melhor parâmetro
kfold = StratifiedKFold(n_splits=5, shuffle=True)
grid_search = GridSearchCV(dt, param_grid, scoring='f1', n_jobs=-1, cv=kfold)
grid_result = grid_search.fit(X_train, y_train)

# ver resultados
print("Melhor: {} para {}".format(grid_result.best_score_, grid_result.best_params_))


Melhor: 0.960039066776428 para {'criterion': 'gini'}


In [ ]:
dt = DecisionTreeClassifier(criterion = 'entropy', max_depth = 8)

param_grid = {
 'splitter':['best','randam']
}

# identificar melhor parâmetro
kfold = StratifiedKFold(n_splits=5, shuffle=True)
grid_search = GridSearchCV(dt, param_grid, scoring='f1', n_jobs=-1, cv=kfold)
grid_result = grid_search.fit(X_train, y_train)

# ver resultados
print("Melhor: {} para {}".format(grid_result.best_score_, grid_result.best_params_))

Melhor: 0.9450388923441473 para {'splitter': 'best'}


In [ ]:
dt = DecisionTreeClassifier(criterion = 'entropy',splitter='best', max_depth = 8)

dt.fit(X_train,y_train)

y_pred = dt.predict(X_test)

# Classification Report
print(classification_report(y_test, y_pred))

# imprimir a área sob a curva
print("AUC: {:.4f}\n".format(roc_auc_score(y_test, y_pred)))

              precision    recall  f1-score   support

           0       0.97      0.98      0.97       607
           1       0.97      0.97      0.97       533

    accuracy                           0.97      1140
   macro avg       0.97      0.97      0.97      1140
weighted avg       0.97      0.97      0.97      1140

AUC: 0.9725



> Aqui podemos observar os resultados, como esperado na hipótese inicial o modelo de arvore de decisão não desempenho como o modelo de SVM, porém superou as expectivas, pois teve uma taxa de acerto superior a 95%. Isso é um bom indicador que a previsão feita pelo modelo SVM esteja correta.

> Vamos agora verificar os resultados aplicando o modelo de Regressão Logistica:

## <font color = 'red'> 5.2 Modelo de Regressão Logistica : <font>

> Nossa hipótese inicial aqui é que o desempenho desse modelo fique entre os dois previstos anteriormente, algo por volta dos 97% de acerto. Vamos setar os hiperametros para maximar o resultado do modelo, e eles são:

* `C`

* `solver`

* `penalty`

In [ ]:
model_RL = LogisticRegression(C = 1)
param_grid = {
 'penalty':['L1','L2','elastic','none']
}

# identificar melhor parâmetro
kfold = StratifiedKFold(n_splits=5, shuffle=True)
grid_search = GridSearchCV(model_RL, param_grid, scoring='f1', n_jobs=-1, cv=kfold)
grid_result = grid_search.fit(X_train, y_train)

# ver resultados
print("Melhor: {} para {}".format(grid_result.best_score_, grid_result.best_params_))

Melhor: 0.9797694152841057 para {'penalty': 'none'}


In [ ]:
model_RL = LogisticRegression(penalty = 'none')
param_grid = {
  'C':[0.00001, 0.0001, 0.001, 0.01, 0.1, 1, 10, 100, 1000]
}

# identificar melhor parâmetro
kfold = StratifiedKFold(n_splits=5, shuffle=True)
grid_search = GridSearchCV(model_RL, param_grid, scoring='f1', n_jobs=-1, cv=kfold)
grid_result = grid_search.fit(X_train, y_train)

# ver resultados
print("Melhor: {} para {}".format(grid_result.best_score_, grid_result.best_params_))

Melhor: 0.9789275567516723 para {'C': 1e-05}


In [ ]:
model_RL = LogisticRegression(C = 0.00001, penalty = 'none',)
param_grid = {
  'solver':['newton-cg', 'lbfgs', 'liblinear', 'sag', 'saga']
}

# identificar melhor parâmetro
kfold = StratifiedKFold(n_splits=5, shuffle=True)
grid_search = GridSearchCV(model_RL, param_grid, scoring='f1', n_jobs=-1, cv=kfold)
grid_result = grid_search.fit(X_train, y_train)

# ver resultados
print("Melhor: {} para {}".format(grid_result.best_score_, grid_result.best_params_))

Melhor: 0.9982832618025752 para {'solver': 'newton-cg'}


In [ ]:
model_RL = LogisticRegression(C = 0.00001, penalty = 'none', solver = 'newton-cg')
model_RL.fit(X_train,y_train)

y_pred = model_RL.predict(X_test)

# Classification Report
print(classification_report(y_test, y_pred))

# imprimir a área sob a curva
print("AUC: {:.4f}\n".format(roc_auc_score(y_test, y_pred)))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00       607
           1       1.00      1.00      1.00       533

    accuracy                           1.00      1140
   macro avg       1.00      1.00      1.00      1140
weighted avg       1.00      1.00      1.00      1140

AUC: 0.9982



> Como podemos observar nos resultados, esses foram melhores do que o esperado na hipótese inicial de 97%, quase alcançando o resultado do modelo de SVM, atingindo a marca de 99% de acerto.

## <font color = 'red'> 6. Conclusão : <font>

 
 Vamos aos pricipais pontos:

  * Ao contrário da hipótese inicial de que os dados poderiam influênciar negativamente o desempenho do modelo, vimos que não, o desempenho do modelo foi o melhor possível.

  * O modelo se SVM provou ser um ótimo modelo para o projeto. Os outros modelos mostraram que o desempenho do modelo de SVM de 100% de acerto é um desempenho plausível.

  * A base de dados utilizada nesse notebook de alguma forma potencializou o desempenho dos modelos.

  * Cabe um estudo para verificar a diferença entre esse dados e os dados usados a priori no projeto, uma vez que teoricamente são o mesmo tipo de dado. Minha hipótese é que os dados foram organizados de forma diferente.